In [5]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from Import_Data import import_by_name

#names
# BCW683           10
# busvan-445       8
# chorales-107     27
# lymphography142  5
# monks-train-124  24 

name = 'busvan-445'

A, number_of_row, number_of_continuous = import_by_name(name)
row_A, col_A = A.shape
Mz = -1000   # big-M coefficient

def generate_sub_support(m, x, b, row, sol_index):
    #---------------------------------------------------
    #This function receive infeasible model returns a cut
    #
    support_all = []
    x.ub = np.inf * np.ones(len(row))
    for i in range(-1, len(row)):
        if i != -1:
            x.ub = np.inf * np.ones(len(row))
            x[i].ub = 0
        m.optimize()
        status = m.status
        if status == GRB.OPTIMAL:
            # print('The optimal objective is %g' % m.objVal)
            support = list(map(lambda x: x[1] if x[0] != 0 else None, zip(x.x, [i for i in range(len(b))])))
            support = list(
                map(lambda x: (sol_index + [i for i in range(number_of_row, row_A)])[x] if x != None else None,
                    support))
            support = list(filter(lambda x: x != None and x < number_of_row, support))
            if (support not in support_all) and support !=[]:
                support_all.append(support)
        if status == GRB.INF_OR_UNBD or status == GRB.INFEASIBLE:
            # print('Optimization was stopped with status %d' % status)
            pass
    return support_all


def generate_support(sol):
    #---------------------------------------
    #This function receive fractional solution and return a set of cuts' indeices
    #sol: the fractional solution
    #support_all: the set of cuts' indeices. each row in support_all represents a cuts non-zero element index
    #---------------------------------------
    combination_filter = list(zip(*filter(lambda x: x[0]==1 or x[0]==0, zip(sol, [i for i in range(len(sol))]))))
    # Get the integer(0 or 1) index of a solution
    sol = list(combination_filter[0])
    # sol[i]=1 of sol[i]=0 or 1 otherwise 0
    sol_index = list(combination_filter[1])
    # index
    
    row = sol_index + [i for i in range(number_of_row, row_A)]
    k = 2
    b = np.array(list(map(lambda x: -Mz * x, sol))).reshape(-1, 1)
    b = np.vstack((b, np.ones((row_A - number_of_row - 1, 1))))
    b = np.vstack((b, -0.001))
    w = np.array([k for i in range(len(sol_index))] + [1 for i in range(number_of_row, row_A)])
    # prepapre for IIS compute

    m = gp.Model()
    x = m.addMVar(shape=len(row), lb=0, vtype=GRB.CONTINUOUS, name="x")
    m.addConstr(A[row].T @ x == 0, name="c")
    m.addConstr(b.reshape(-1) @ x == -1, name='b')
    m.setObjective(w @ x, GRB.MINIMIZE)
    m.setParam('OutputFlag', 0)
    # IIS model
    support_all = generate_sub_support(m, x, b, row, sol_index)
    return support_all


def mycallback(model, where):
    if where == GRB.Callback.MIPSOL:
        sol = model.cbGetSolution(model._vars)
        sol=[round(i) for i in sol]
        support_all = generate_support(sol)
        for support in support_all:
            model.cbLazy(
                gp.quicksum(model._vars[i] if sol[i] == 0 else 0
                            for i in support) +
                gp.quicksum((1 - model._vars[i]) if sol[i] == 1 else 0
                            for i in support) >= 1)
    elif where == GRB.Callback.MIPNODE:
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL and model.cbGet(GRB.Callback.RUNTIME)<60:
            sol = model.cbGetNodeRel(model._vars)
            support_all = generate_support(sol)
            for support in support_all:
                model.cbLazy(
                    gp.quicksum(model._vars[i] if sol[i] == 0 else 0
                                for i in support) +
                    gp.quicksum((1 - model._vars[i]) if sol[i] == 1 else 0
                                for i in support) >= 1)
    return 0


# Experiments 

master = gp.Model(name)
z = master.addVars(number_of_row, vtype=GRB.BINARY, name='z')
master.setObjective(gp.quicksum(z[i] for i in range(number_of_row)), GRB.MINIMIZE)
# initially empty master problem

master.update()
master._vars = master.getVars()
master.Params.lazyConstraints = 1
master.Params.ZeroHalfCuts = 2
master.optimize(mycallback)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter ZeroHalfCuts to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 445 columns and 0 nonzeros
Model fingerprint: 0x0034bc22
Variable types: 0 continuous, 445 integer (445 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 445 columns, 0 nonzeros
Variable types: 0 continuous, 445 integer (445 binary)

Root relaxation: objective 1.333333e+00, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.33333    0    4          -    1.